# Master Thesis - Particle Swarm Optimization with constraints - Implementation (Numerical Scheme)

In [6]:
import numpy as np

The equations of motion for particle swarm optimization are obtained by adding the interaction, friction, exploration, and constraint terms and this can be written in continuous time formulation as
\begin{equation}
\begin{aligned}
    dX^i(t) &= V^i(t)dt \\
    dV^i(t) &= -\lambda(X^i(t)-\overline{X}(t))dt - \gamma V_i(t)dt+\sigma(t)dB^i(t) +dR(t),
\end{aligned}
\end{equation}
where $dR(t)$ is defined as 
\begin{equation}
    dR(t) = -2\braket{n(X^i(t)),V^i(t)}n(X^i(t))I_{\partial G}(X(t)),
\end{equation}
 $\lambda$ and $\gamma$ are hyper-parameters which will be tested in a range and evaluated using a success rate 
\begin{equation}
    ||X_{min}-X^*|| < 0.2
\end{equation}
Where $X^*$ is the collapsed ensamble value of the positions. Collapsed means that all particles have converged to the same position, which is inevitable if the global minimum has been found due to the interaction term.

To use these SDEs for numerical simulations they have to be rewritten in discrete time formulation. This is done by way of Euler-Maruyama scheme shown in equation \eqref{eq:discrete} to obtain the expression for the position and velocity seen in equation \eqref{eq:PSO_discrete}

\begin{equation}
\begin{aligned}
    X_{k+1}^i &= X_k^i + V_k^ih, \\
    V_{k+1}^i &= V_k^i -\lambda(X^i_k-\overline{X}_k)h - \gamma V^i_kh+\sigma_k\sqrt{h}\xi_{k+1}^i+dR_k,
\end{aligned}
\end{equation}
note that $\sigma_k = \sigma(t_k)$, and


In [7]:
T = 5 # Total time
h = 0.01 # time step
N = int(T/h) # number of time steps, 500
N_particles = 100
X = np.zeros((N_particles,2))

PSO is driven by SDE that contains four terms: an interaction term that looks like 
\begin{equation}
    -\int_0^t\lambda(X^i(s)-\overline{X}(s))ds,
\end{equation} 
where $X^i(s)$ is the position of a particle, i, at time, t and $\overline{X}(s)$ is a weighted average of all particles at a time t. The weighted average is defined as
\begin{equation}
    \overline{X}(t) = \sum_{j=1}^N \frac{X^j(t)e^{-\alpha U(X^j(t))}}{\sum_{l=1}^Ne^{-\alpha U(X^l(t))}} = \left[\frac{e^{-\alpha U(X^j(t))}}{\sum_{l=1}^Ne^{-\alpha U(X^l(t))}} \equiv w^j\right] = \sum_{j=1}^N X^j(t)w^j,
\end{equation}

In [8]:
def weighted_average(X,alpha,objective_function):
    total_weight = np.sum(np.exp(-alpha*objective_function(X)))
    w = np.exp(-alpha*objective_function(X))/total_weight
    return np.sum(X*w)

$dR_k$ is the reflective term in discrete time formulation.

IF $\hat{X}^i_{k+1} \in G^C$:
\begin{equation}
\begin{aligned}
X^i_{\tau,k+1} &= X^i_k+\tau V^i_k \in \partial G, [\tau \in (0,h)] \\
\hat{V}^i_k &= V^i_k -2\braket{ n(X^i_{\tau,k+1}),V^i_k} n(X^i_{\tau,k+1}) \\
X^i_{k+1} &= X^i_{\tau,k+1}+(h-\tau)\hat{V}^i_k
\end{aligned}
\end{equation} 

ELSE:
\begin{equation}
\begin{aligned}
X^i_{k+1} &= \hat{X}^i_{k+1} \\
\hat{V}^i_k &= V^i_k
\end{aligned}
\end{equation}

and the iterative update of the discrete velocity is
\begin{equation}
    V^i_{k+1} = \hat{V}^i_k -\lambda(X^i_{k+1}-\overline{X}_{k+1})h - \gamma \hat{V}^i_k h + \sigma_k\xi^i_{k+1}\sqrt{h}.
\end{equation}

For this function i need to return $-2\braket{ n(X^i_{\tau,k+1}),V^i_k} n(X^i_{\tau,k+1})$, if $X_{new}$ went into $G^C$, and 0 otherwise

In [ ]:
def Reflection_spherical(X_old,V, radius): # actually circular Centered at the origin
    X_new = X_old + h*V
    norms = np.linalg.norm(X_new,axis=1)
    reflect_mask = norms > radius
    if np.any(reflect_mask):
        #First step is finding the intersection point of the line and the sphere, aka tau
        a_1 = X_new[0]
        a_2 = X_new[1]
        b_1 = V[0]
        b_2 = V[1]
        # Must solve (b_1**2+b_2**2)tau**2 + 2*(a_1*b_1+a_2*b_2)*tau + (a_1**2+a_2**2-radius**2) = 0
        p = 2*(a_1*b_1+a_2*b_2)/(b_1**2+b_2**2)
        q = (a_1**2+a_2**2-radius**2)/(b_1**2+b_2**2)
        tau = -p/2 + np.sqrt(p**2/4 - q)
        X_tau = X_old + tau*V
        outward_normal = X_tau/np.linalg.norm(X_tau) 
        #the outward normal is calculated as the normalized position vector of the intersection point X_tau_i.
        #This works because for a circular (or spherical) boundary centered at the origin, 
        # the outward normal at any point on the boundary is simply the unit vector pointing from the origin to that point.
        V_hat_i = V - 2*np.dot(outward_normal,V)*outward_normal
        X_new = X_tau + (h-tau)*V_hat_i
        return X_new, V_hat_i
    else:
        return X_new, V

In [ ]:
import numpy as np

def Reflection_spherical(X_old, V, radius, h):
    N_particles = X_old.shape[0]
    X_new = X_old + h * V
    
    # Calculate norms for all particles
    norms = np.linalg.norm(X_new, axis=1)
    
    # Identify particles that need reflection
    reflect_mask = norms > radius
    
    if np.any(reflect_mask):
        a = X_old[reflect_mask]
        b = V[reflect_mask]
        
        # Vectorized calculations
        p = 2 * np.sum(a * b, axis=1) / np.sum(b**2, axis=1)
        q = (np.sum(a**2, axis=1) - radius**2) / np.sum(b**2, axis=1)
        tau = -p/2 + np.sqrt(p**2/4 - q)
        
        X_tau = a + tau[:, np.newaxis] * b
        
        # Calculate outward normals
        outward_normal = X_tau / np.linalg.norm(X_tau, axis=1)[:, np.newaxis]
        
        # Calculate reflected velocities
        V_hat = b - 2 * np.sum(b * outward_normal, axis=1)[:, np.newaxis] * outward_normal
        
        # Update positions for reflected particles
        X_new[reflect_mask] = X_tau + (h - tau[:, np.newaxis]) * V_hat
        V[reflect_mask] = V_hat
    
    return X_new, V


In [ ]:
def PSO(X_old,weighted_average_X_old, V_old, Lambda, gamma, sigma):
    # X: particle positions
    # weighted_average_X: weighted average of particle positions
    # V: particle velocities
    # Lambda: interaction coefficient
    # gamma: inertia coefficient
    # sigma: exploration coefficient
    # returns updated X, weighted_average_X, V
    
    # update velocity
    X_hat_new, V_hat_new = Reflection_spherical(X_old)
    X_new = X_old + V_old*h
    V_new = V_old - Lambda*(X_old-weighted_average_X_old)*h-gamma*V_old*h+sigma*np.sqrt(h)*np.random.randn(N,1)
    
    # update position
    X = X + V
    
    return X, V